In [1]:
import csv
import pandas as pd 
import torch
import torch.nn as nn


In [2]:
from datasets import load_dataset

# s='train[:10]'
# dataset_train = load_dataset('csv', data_files='/kaggle/input/translated-data/Translated_hindi_train.csv')
# dataset_val = load_dataset('csv', data_files='/kaggle/input/summariser-data2022/val.csv')

/home/nikita21546/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# dataset_train

In [4]:
df_train = pd.read_csv('Translated_hindi_train.csv')     
# df_val = pd.read_csv('/kaggle/input/summariser-data2022/val.csv')     

In [5]:
article_limit = 300
# summary_limit=
def resize(article,summary):
    a=[]
    s=[]
    for i in range(len(article)):
        if(len(article[i].split())>article_limit):
            a.append(article[i])
            s.append(summary[i])
        else:
            continue
    
    return a,s


In [6]:
train_article = df_train['Article'].tolist()[:2000]
train_summary = df_train['Summary'].tolist()[:2000]
# train_article,train_summary = resize(train_article,train_summary)

In [7]:
len(train_article)

2000

In [8]:
# from torch.utils.data import Dataset
from datasets import DatasetDict, Dataset

x=100

t=int(len(train_article)*0.9)
print(t)

data_dict = {
    "Article": train_article[:t],
    "Summary": train_summary[:t]
}

# Create a Dataset object from the dictionary
dataset_train = Dataset.from_dict(data_dict)

dict_train = DatasetDict({
    'train': dataset_train,
    # Add more datasets for validation, test, etc., as needed
})

print((dict_train))
# print(t)

1800
DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary'],
        num_rows: 1800
    })
})


In [9]:

# val_article = df_val['Article'].tolist()[:x]
# val_summary = df_val['Summary'].tolist()[:x]

val_article = train_article[t:]
val_summary = train_summary[t:] 

# val_article,val_summary = resize(val_article,val_summary)

data_dict = {
    "Article": val_article,
    "Summary": val_summary
}

# Create a Dataset object from the dictionary
dataset_val = Dataset.from_dict(data_dict)

dict_val = DatasetDict({
    'validation': dataset_val,
    # Add more datasets for validation, test, etc., as needed
})

print(type(dict_val))
dict_val

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    validation: Dataset({
        features: ['Article', 'Summary'],
        num_rows: 200
    })
})

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
# model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Someman/bart-hindi")
# model = AutoModelForSeq2SeqLM.from_pretrained("Someman/bart-hindi")

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/nikita21546/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [11]:
# from transformers import AutoTokenizer
# import torch

# df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')
# text= df_test['Article'].tolist()[12]

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# inputs = tokenizer(text, return_tensors="pt")

# # Move the input tensors to the same device as the model
# inputs = {k: v.to(device) for k, v in inputs.items()}

# # Generate outputs using the model
# with torch.no_grad():
#     # Access the correct key for `input_ids` in the `inputs` dictionary
#     input_ids = inputs["input_ids"]
#     outputs = model.generate(input_ids, max_new_tokens=70, do_sample=False)

# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print("Generated Text:", generated_text)


In [12]:
# summary = df_test['Summary'].tolist()[12]
# summary

In [13]:
# text

In [14]:
max_input_length = 600
max_target_length = 40

prefix="summarize: "
x=10
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True,padding='max_length')
#     labels = tokenizer(text_target=examples["Summary"], padding='longest', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
# inp = preprocess_function(dataset_train['train'])

# x=10
# tokenised_data_train = dataset_train.map(preprocess_function, batched=True)
# tokenised_data_val = dataset_val.map(preprocess_function, batched=True)

tokenised_data_train = dict_train.map(preprocess_function, batched=True)
tokenised_data_val = dict_val.map(preprocess_function, batched=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 2062.43 examples/s]


In [16]:
(tokenised_data_train)

DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1800
    })
})

In [17]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='mT5_multilingual_XLSum')

In [19]:
# !pip install rouge_score
# !pip install evaluate

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

# rouge = evaluate.load("rouge")


batch_size = 1
# model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="mT5_multilingual_XLSum",
#     eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    save_steps=500  ,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
torch.cuda.empty_cache()

In [22]:
tokenised_data_train


DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1800
    })
})

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenised_data_train["train"],
    eval_dataset=tokenised_data_val["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
#      data_collator=data_collator,
)

trainer.train()

/home/nikita21546/.local/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,2.261300
1000,2.192800
1500,2.021100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-

TrainOutput(global_step=1800, training_loss=2.104666002061632, metrics={'train_runtime': 293.6417, 'train_samples_per_second': 6.13, 'train_steps_per_second': 6.13, 'total_flos': 2529237934080000.0, 'train_loss': 2.104666002061632, 'epoch': 1.0})

In [24]:
print()

In [25]:
text = df_train['Article'].tolist()[11]
text='summarise: '+text
text

'summarise: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट नौकरियों की जानकारी के साथ हाजिर हैं। राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी है। आवेदन की शुरुआत एक अगस्त 2023 से होगी।\nरेल इंडिया टेक्निकल एंड इकोनॉमिक सर्विस (राइट्स) ने भारत के रेल मंत्रालय के तहत ड्राफ्ट्समैन सिविल इंजीनियर, पर्यावरण सामाजिक निगरानी विशेषज्ञ, जूनियर डिजाइन इंजीनियर के 111 पदों के लिए ऑनलाइन आवेदन शुरू कर दिया है। 7 अगस्त तक ऑनलाइन आवेदन कर सकते हैं।\nभुवनेश्वर एम्स में सीनियर नर्सिंग ऑफिसर, स्टोर कीपर, कैशियर, कनिष्ठ प्रशासनिक सहायक, मेडिकल रिकॉर्ड तकनीशियन (रिकॉर्ड क्लर्क), वायरमैन, फार्मासिस्ट के 775 पदों के लिए वैकेंसी थी। अगर आप अभी तक अप्लाई नहीं कर सके हैं तो आज यानी 30 जुलाई लास्ट डेट है।\nभू यानी बनारस हिंदू विश्वविद्यालय ने असिस्टेंट प्रोफेसर, एसोसिएट प्रोफेसर और प्रोफेसर के 307 पदों पर भर्ती निकाली है। अप्लाई करने की लास्ट डेट कल यानी 31 जुलाई 2023 है।\nनेत्र परीक्षण अधिकारी के 157 पदों पर भर्ती निकली है। आवेदन की आखिरी डेट 7 अगस्त 2023 है। सिलेक्शन होने पर 25,

In [26]:
# Save tokenizer
import pickle
with open('/kaggle/working/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/tokenizer.pkl'

In [52]:
import pickle

# Save model
with open('mt5_xl_23_translated_modelN.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save tokenizer
with open('mt5_xl_23_translated_tokenizerN.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [29]:
import pandas as pd

df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/summariser-data2022/test.csv'

In [ ]:
from transformers import pipeline
# prin
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
text= df_test['Article'].tolist()[12]
summarizer = pipeline("summarization", model=model,tokenizer=AutoTokenizer.from_pretrained("/kaggle/working/tokeniser"))
summarizer(text)

In [30]:
text

'summarise: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट नौकरियों की जानकारी के साथ हाजिर हैं। राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी है। आवेदन की शुरुआत एक अगस्त 2023 से होगी।\nरेल इंडिया टेक्निकल एंड इकोनॉमिक सर्विस (राइट्स) ने भारत के रेल मंत्रालय के तहत ड्राफ्ट्समैन सिविल इंजीनियर, पर्यावरण सामाजिक निगरानी विशेषज्ञ, जूनियर डिजाइन इंजीनियर के 111 पदों के लिए ऑनलाइन आवेदन शुरू कर दिया है। 7 अगस्त तक ऑनलाइन आवेदन कर सकते हैं।\nभुवनेश्वर एम्स में सीनियर नर्सिंग ऑफिसर, स्टोर कीपर, कैशियर, कनिष्ठ प्रशासनिक सहायक, मेडिकल रिकॉर्ड तकनीशियन (रिकॉर्ड क्लर्क), वायरमैन, फार्मासिस्ट के 775 पदों के लिए वैकेंसी थी। अगर आप अभी तक अप्लाई नहीं कर सके हैं तो आज यानी 30 जुलाई लास्ट डेट है।\nभू यानी बनारस हिंदू विश्वविद्यालय ने असिस्टेंट प्रोफेसर, एसोसिएट प्रोफेसर और प्रोफेसर के 307 पदों पर भर्ती निकाली है। अप्लाई करने की लास्ट डेट कल यानी 31 जुलाई 2023 है।\nनेत्र परीक्षण अधिकारी के 157 पदों पर भर्ती निकली है। आवेदन की आखिरी डेट 7 अगस्त 2023 है। सिलेक्शन होने पर 25,

In [31]:
t=df_test['Summary'].tolist()[12]
t

NameError: name 'df_test' is not defined

In [32]:
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Indic_Bart.pkl'

In [33]:
from transformers import AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Generated Text: राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी


In [34]:
text

'summarise: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट नौकरियों की जानकारी के साथ हाजिर हैं। राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी है। आवेदन की शुरुआत एक अगस्त 2023 से होगी।\nरेल इंडिया टेक्निकल एंड इकोनॉमिक सर्विस (राइट्स) ने भारत के रेल मंत्रालय के तहत ड्राफ्ट्समैन सिविल इंजीनियर, पर्यावरण सामाजिक निगरानी विशेषज्ञ, जूनियर डिजाइन इंजीनियर के 111 पदों के लिए ऑनलाइन आवेदन शुरू कर दिया है। 7 अगस्त तक ऑनलाइन आवेदन कर सकते हैं।\nभुवनेश्वर एम्स में सीनियर नर्सिंग ऑफिसर, स्टोर कीपर, कैशियर, कनिष्ठ प्रशासनिक सहायक, मेडिकल रिकॉर्ड तकनीशियन (रिकॉर्ड क्लर्क), वायरमैन, फार्मासिस्ट के 775 पदों के लिए वैकेंसी थी। अगर आप अभी तक अप्लाई नहीं कर सके हैं तो आज यानी 30 जुलाई लास्ट डेट है।\nभू यानी बनारस हिंदू विश्वविद्यालय ने असिस्टेंट प्रोफेसर, एसोसिएट प्रोफेसर और प्रोफेसर के 307 पदों पर भर्ती निकाली है। अप्लाई करने की लास्ट डेट कल यानी 31 जुलाई 2023 है।\nनेत्र परीक्षण अधिकारी के 157 पदों पर भर्ती निकली है। आवेदन की आखिरी डेट 7 अगस्त 2023 है। सिलेक्शन होने पर 25,

In [35]:
t=df_test['Summary'].tolist()[12]
t

NameError: name 'df_test' is not defined

In [36]:
text

'summarise: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट नौकरियों की जानकारी के साथ हाजिर हैं। राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी है। आवेदन की शुरुआत एक अगस्त 2023 से होगी।\nरेल इंडिया टेक्निकल एंड इकोनॉमिक सर्विस (राइट्स) ने भारत के रेल मंत्रालय के तहत ड्राफ्ट्समैन सिविल इंजीनियर, पर्यावरण सामाजिक निगरानी विशेषज्ञ, जूनियर डिजाइन इंजीनियर के 111 पदों के लिए ऑनलाइन आवेदन शुरू कर दिया है। 7 अगस्त तक ऑनलाइन आवेदन कर सकते हैं।\nभुवनेश्वर एम्स में सीनियर नर्सिंग ऑफिसर, स्टोर कीपर, कैशियर, कनिष्ठ प्रशासनिक सहायक, मेडिकल रिकॉर्ड तकनीशियन (रिकॉर्ड क्लर्क), वायरमैन, फार्मासिस्ट के 775 पदों के लिए वैकेंसी थी। अगर आप अभी तक अप्लाई नहीं कर सके हैं तो आज यानी 30 जुलाई लास्ट डेट है।\nभू यानी बनारस हिंदू विश्वविद्यालय ने असिस्टेंट प्रोफेसर, एसोसिएट प्रोफेसर और प्रोफेसर के 307 पदों पर भर्ती निकाली है। अप्लाई करने की लास्ट डेट कल यानी 31 जुलाई 2023 है।\nनेत्र परीक्षण अधिकारी के 157 पदों पर भर्ती निकली है। आवेदन की आखिरी डेट 7 अगस्त 2023 है। सिलेक्शन होने पर 25,

In [37]:
!pip install indic-nlp-library


/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
     |████████████████████████████████| 40 kB 1.2 MB/s eta 0:00:01


In [38]:
from transformers import AutoTokenizer
import torch

# with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
#     model = pickle.load(f)
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# def summarise(s):

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=80, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# return generated_text
print("Generated Text:", generated_text)

Generated Text: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट


In [39]:
summaries = []
ori_summary = []
s=df_test['Summary'].tolist()
a = df_test['Article'].tolist()
print(len(s))
ori_summary.append(s[0])
print(a[0])
# summaries.append(summarise(a[0]))
# for i in range(len(s)):
#     ori_summary.append(s[i])
#     summaries.append(summarise(a[i]))

NameError: name 'df_test' is not defined

In [ ]:
!pip install rouge

In [ ]:
df_train=pd.read_csv('Translated_hindi_train.csv')

df_test2=df_train

In [48]:

def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

df_test2['predicted_summary'] = df_test2['Article'].apply(predict_summary)

<ipython-input-48-64a1bdbb80e9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test2['predicted_summary'] = df_test2['Article'].apply(predict_summary)


In [51]:
from rouge import Rouge

rouge = Rouge()


def calculate_rouge(row):
    return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

df_test2['rouge'] = df_test2.apply(calculate_rouge, axis=1)
df_test2['rouge-1'] = df_test2['rouge'].apply(lambda x: x['rouge-1']['f'])
df_test2['rouge-2'] = df_test2['rouge'].apply(lambda x: x['rouge-2']['f'])
df_test2['rouge-l'] = df_test2['rouge'].apply(lambda x: x['rouge-l']['f'])

print("Rouge-1 :",df_test2['rouge-1'].mean())
print("Rouge-2 :",df_test2['rouge-2'].mean())
print("Rouge-L :",df_test2['rouge-l'].mean())

Rouge-1 : 0.3409268450872674
Rouge-2 : 0.23242517666452361
Rouge-L : 0.32349552488193495


<ipython-input-51-aaea770f66c2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test2['rouge'] = df_test2.apply(calculate_rouge, axis=1)
<ipython-input-51-aaea770f66c2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test2['rouge-1'] = df_test2['rouge'].apply(lambda x: x['rouge-1']['f'])
<ipython-input-51-aaea770f66c2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [ ]:
!pip install rouge

In [ ]:
# # Define compute_metrics function
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Load ROUGE and BERT score metrics
#     rouge = load_metric('rouge')
#     bert_score = load_metric('bertscore')

#     # Compute ROUGE and BERT scores
#     rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rougeL", "rouge2"])
#     bert_score_output = bert_score.compute(predictions=decoded_preds, references=decoded_labels, lang="hi")

#     return {
#         "rouge2": rouge_output['rouge2'].mid.fmeasure,
#         "rougeL": rouge_output['rougeL'].mid.fmeasure,
#         "bert_score": bert_score_output['f1'][0]
#     }